In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm
import joblib

# Scikit-image pour les textures
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from skimage import filters

# Sklearn pour le ML
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier

# XGBoost
import xgboost as xgb
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/deepglobe-land-cover-classification-dataset/class_dict.csv
/kaggle/input/deepglobe-land-cover-classification-dataset/metadata.csv
/kaggle/input/deepglobe-land-cover-classification-dataset/valid/8285_sat.jpg
/kaggle/input/deepglobe-land-cover-classification-dataset/valid/127801_sat.jpg
/kaggle/input/deepglobe-land-cover-classification-dataset/valid/198275_sat.jpg
/kaggle/input/deepglobe-land-cover-classification-dataset/valid/393509_sat.jpg
/kaggle/input/deepglobe-land-cover-classification-dataset/valid/262504_sat.jpg
/kaggle/input/deepglobe-land-cover-classification-dataset/valid/651633_sat.jpg
/kaggle/input/deepglobe-land-cover-classification-dataset/valid/392604_sat.jpg
/kaggle/input/deepglobe-land-cover-classification-dataset/valid/411176_sat.jpg
/kaggle/input/deepglobe-land-cover-classification-dataset/valid/808681_sat.jpg
/kaggle/input/deepglobe-land-cover-classification-dataset/valid/259219_sat.jpg
/kaggle/input/deepglobe-land-cover-classification-dataset/valid/6863

In [2]:
# Configuration
BASE_PATH = '/kaggle/input/deepglobe-land-cover-classification-dataset/'
TRAIN_PATH = os.path.join(BASE_PATH, 'train')
CLASS_NAMES = ['urban', 'agriculture', 'rangeland', 'forest', 'water', 'barren',]

# Mapping Couleurs (RGB) vers Label (0-5)
class_dict = pd.read_csv(os.path.join(BASE_PATH, 'class_dict.csv'))
LABEL_TO_COLOR = {i: tuple(class_dict.iloc[i, 1:4].values) for i in range(6)}
COLOR_TO_LABEL = {v: k for k, v in LABEL_TO_COLOR.items()}

print("Environnement prêt. Classes :", CLASS_NAMES)

Environnement prêt. Classes : ['urban', 'agriculture', 'rangeland', 'forest', 'water', 'barren']


In [3]:
def load_image_and_mask(img_id):
    """Charge une image et son masque converti en labels 0-5"""
    path_img = os.path.join(TRAIN_PATH, f"{img_id}_sat.jpg")
    path_mask = os.path.join(TRAIN_PATH, f"{img_id}_mask.png")
    
    img = np.array(Image.open(path_img))
    mask_rgb = np.array(Image.open(path_mask))
    
    # Conversion RGB -> Label rapide
    mask = np.zeros(mask_rgb.shape[:2], dtype=np.uint8)
    # Note: Cette méthode simple suppose que les pixels correspondent exactement
    # Pour une méthode robuste vectorisée:
    r, g, b = mask_rgb[:,:,0], mask_rgb[:,:,1], mask_rgb[:,:,2]
    for label_id, color in LABEL_TO_COLOR.items():
        # Création d'un masque booléen pour chaque couleur
        mask_match = (r == color[0]) & (g == color[1]) & (b == color[2])
        mask[mask_match] = label_id
        
    return img, mask

class OptimizedFeatureExtractor:
    def __init__(self):
        self.distances = [1, 3]
        self.angles = [0, np.pi/4, np.pi/2]
    
    def extract(self, patch_rgb):
        # Conversion float pour calculs
        patch = patch_rgb.astype(np.float32)
        gray = cv2.cvtColor(patch_rgb, cv2.COLOR_RGB2GRAY)
        
        features = []
        
        # 1. Stats Couleur (Mean, Std, Percentiles) - 12 features
        for i in range(3): # R, G, B
            channel = patch[:, :, i]
            features.extend([
                channel.mean(), channel.std(),
                np.percentile(channel, 10), np.percentile(channel, 90)
            ])
        
        # 2. GLCM (Texture) - 8 features
        gray8 = (gray / 32).astype(np.uint8) # Réduction à 8 niveaux de gris
        glcm = graycomatrix(gray8, distances=self.distances, angles=self.angles,
                           levels=8, symmetric=True, normed=True)
        for prop in ['contrast', 'homogeneity', 'energy', 'correlation']:
            vals = graycoprops(glcm, prop).flatten()
            features.extend([vals.mean(), vals.std()])
        
        # 3. LBP (Motifs locaux) - 10 features
        lbp = local_binary_pattern(gray, P=8, R=1, method='uniform')
        hist, _ = np.histogram(lbp, bins=10, range=(0, 10), density=True)
        features.extend(hist)
        
        # 4. Gradient (Sobel) - 2 features
        sx = filters.sobel(gray, axis=0)
        sy = filters.sobel(gray, axis=1)
        grad_mag = np.sqrt(sx**2 + sy**2)
        features.extend([grad_mag.mean(), grad_mag.std()])
        
        return np.array(features, dtype=np.float32)

extractor = OptimizedFeatureExtractor()
print("Extracteur initialisé.")

Extracteur initialisé.


In [4]:
def extract_balanced_patches(n_images=250, patch_size=64, patches_per_class=3200):
    train_ids = sorted({f.split('_')[0] for f in os.listdir(TRAIN_PATH) if f.endswith('_sat.jpg')})
    selected_ids = np.random.choice(train_ids, n_images, replace=False)
    
    X, y = [], []
    class_count = {i: 0 for i in range(6)}
    
    print(f"Extraction sur {n_images} images...")
    
    for img_id in tqdm(selected_ids):
        # Arrêt si on a assez de données pour toutes les classes
        if all(c >= patches_per_class for c in class_count.values()):
            break
            
        img, mask = load_image_and_mask(img_id)
        h, w = img.shape[:2]
        
        # Tentatives aléatoires pour trouver des patchs de chaque classe
        attempts = 0
        while attempts < 200: # Limite pour ne pas bloquer
            y0 = np.random.randint(0, h - patch_size)
            x0 = np.random.randint(0, w - patch_size)
            
            patch_mask = mask[y0:y0+patch_size, x0:x0+patch_size]
            
            # On prend la classe majoritaire du patch
            counts = np.bincount(patch_mask.flatten(), minlength=7)
            label = counts.argmax()
            
            # On ignore 'unknown' (6) et les classes déjà pleines
            if label != 6 and class_count[label] < patches_per_class:
                patch_img = img[y0:y0+patch_size, x0:x0+patch_size]
                
                X.append(extractor.extract(patch_img))
                y.append(label)
                class_count[label] += 1
            
            attempts += 1
            
    return np.array(X), np.array(y)

# Exécution de l'extraction
X, y = extract_balanced_patches(n_images=250, patch_size=64, patches_per_class=3200)

print("\nDistribution finale des classes :")
unique, counts = np.unique(y, return_counts=True)
for u, c in zip(unique, counts):
    print(f"  {CLASS_NAMES[u]:12} : {c}")

# Nettoyage NaN (sécurité)
X = np.nan_to_num(X)

Extraction sur 250 images...


100%|██████████| 250/250 [02:32<00:00,  1.64it/s]


Distribution finale des classes :
  urban        : 3200
  agriculture  : 3200
  rangeland    : 3200
  forest       : 3200
  water        : 1694
  barren       : 3200


In [5]:
# 1. Split & Scaling
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [6]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# --- 1. Définition des modèles ---
models = {
    "Random Forest": RandomForestClassifier(
        n_estimators=200, 
        max_depth=15, 
        n_jobs=-1, 
        random_state=42
    ),
    
    "XGBoost": xgb.XGBClassifier(
        n_estimators=200,
        max_depth=10,
        learning_rate=0.1,
        tree_method='hist',
        n_jobs=-1,
        random_state=42
    ),
    
    "SVM (RBF)": SVC(
        kernel='rbf', 
        C=1.0, 
        cache_size=1000, 
        random_state=42
    ),
    
    "KNN (k=5)": KNeighborsClassifier(
        n_neighbors=5, 
        n_jobs=-1
    )
}

# --- 2. Boucle d'entraînement et d'évaluation ---
results = []
trained_models = {}
confusion_matrices = {}

print(f"{'Modèle':<15} | {'Train Time':<10} | {'Accuracy':<10} | {'F1-Macro':<10}")
print("-" * 55)

for name, model in models.items():
    # A. Entraînement (Mesure du temps)
    start_train = time.time()
    model.fit(X_train_scaled, y_train)
    train_time = time.time() - start_train
    
    # B. Prédiction
    start_pred = time.time()
    y_pred = model.predict(X_test_scaled)
    pred_time = time.time() - start_pred
    
    # C. Métriques
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='macro')
    
    # Stockage
    results.append({
        "Model": name,
        "Accuracy": acc,
        "F1-Score": f1,
        "Train Time (s)": train_time,
        "Pred Time (s)": pred_time
    })
    
    trained_models[name] = model
    confusion_matrices[name] = confusion_matrix(y_test, y_pred)
    
    print(f"{name:<15} | {train_time:6.2f}s    | {acc:.4f}     | {f1:.4f}")

# Création d'un DataFrame pour l'affichage propre
df_results = pd.DataFrame(results).set_index("Model")

Modèle          | Train Time | Accuracy   | F1-Macro  
-------------------------------------------------------
Random Forest   |   5.17s    | 0.7378     | 0.7476
XGBoost         |  15.14s    | 0.7573     | 0.7695
SVM (RBF)       |   6.79s    | 0.7282     | 0.7386
KNN (k=5)       |   0.00s    | 0.7149     | 0.7288


In [7]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score, accuracy_score
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Métrique principale (tu peux changer selon ton problème)
scoring = {'F1_macro': make_scorer(f1_score, average='macro'),
           'Accuracy': make_scorer(accuracy_score)}

# CV stratifiée (important si classes déséquilibrées)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Espace de recherche élargi et intelligent
param_grid = {
    # Arbre
    'max_depth': [6, 8, 10, 12, 15],
    'min_child_weight': [1, 3, 5, 7],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],

    # Sous-échantillonnage
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bylevel': [0.6, 0.7, 0.8, 0.9, 1.0],

    # Régularisation
    'reg_alpha': [0, 0.001, 0.01, 0.1, 1, 10],
    'reg_lambda': [0.1, 0.5, 1.0, 1.5, 2.0],

    # Apprentissage
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [200, 300, 500, 800, 1000],

    # Gestion du déséquilibre (si besoin)
    # 'scale_pos_weight': [1, (y_train == 0).sum() / (y_train == 1).sum()]  # décommente si binaire déséquilibré
}

# Modèle de base
xgb_model = xgb.XGBClassifier(
    random_state=42,
    n_jobs=-1,
    tree_method='hist',        # plus rapide
    eval_metric='logloss',
    verbosity=0
)

# RandomizedSearch (beaucoup plus rapide que GridSearch sur grand espace)
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=100,                    # ← augmente à 150-200 si tu as le temps
    scoring='f1_macro',            # ou 'accuracy' selon ton objectif
    cv=cv,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

print("Lancement de l'optimisation XGBoost (RandomizedSearchCV)...")
start = time.time()
random_search.fit(X_train_scaled, y_train)
print(f"Optimisation terminée en {time.time() - start:.2f} secondes")

# Meilleurs paramètres
print("\nMEILLEURS PARAMÈTRES TROUVÉS :")
print(random_search.best_params_)
print(f"\nMeilleur F1-macro (CV) : {random_search.best_score_:.4f}")

Lancement de l'optimisation XGBoost (RandomizedSearchCV)...
Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END colsample_bylevel=1.0, colsample_bytree=0.7, gamma=0.1, learning_rate=0.2, max_depth=10, min_child_weight=7, n_estimators=300, reg_alpha=0, reg_lambda=1.0, subsample=1.0; total time=  12.3s
[CV] END colsample_bylevel=1.0, colsample_bytree=0.9, gamma=0.1, learning_rate=0.01, max_depth=6, min_child_weight=7, n_estimators=500, reg_alpha=0.01, reg_lambda=1.0, subsample=0.7; total time=  33.2s
[CV] END colsample_bylevel=1.0, colsample_bytree=0.7, gamma=0.2, learning_rate=0.01, max_depth=8, min_child_weight=1, n_estimators=500, reg_alpha=0.1, reg_lambda=1.0, subsample=0.9; total time= 1.2min
[CV] END colsample_bylevel=1.0, colsample_bytree=0.7, gamma=0.2, learning_rate=0.01, max_depth=8, min_child_weight=1, n_estimators=500, reg_alpha=0.1, reg_lambda=1.0, subsample=0.9; total time= 1.2min
[CV] END colsample_bylevel=0.7, colsample_bytree=0.6, gamma=0.3, learning_

In [8]:
# 1. Show the best  score
print("MEILLEUR SCORE CV (F1-macro) :", random_search.best_score_)
print("\nMEILLEURS PARAMÈTRES :")
print(random_search.best_params_)


MEILLEUR SCORE CV (F1-macro) : 0.7620691839429952

MEILLEURS PARAMÈTRES :
{'subsample': 0.9, 'reg_lambda': 1.0, 'reg_alpha': 0.001, 'n_estimators': 1000, 'min_child_weight': 3, 'max_depth': 15, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.7}


In [9]:
# 2. Re-train the final model on the FULL training set with the best params
best_params = random_search.best_params_
best_params.update({
    'random_state': 42,
    'n_jobs': -1,
    'tree_method': 'hist',
    'eval_metric': 'logloss',
    'verbosity': 0
})

final_xgb = xgb.XGBClassifier(**best_params)
final_xgb.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.7, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=15,
              max_leaves=None, min_child_weight=3, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=1000,
              n_jobs=-1, num_parallel_tree=None, objective='multi:softprob', ...)